### Make sure you have the geoserver VM running

this should show something like:

```
Current machine states:

geoserver                 running (virtualbox)

The VM is running. To stop this VM, you can run `vagrant halt` to
shut it down forcefully, or you can run `vagrant suspend` to simply
suspend the virtual machine. In either case, to restart it again,
simply run `vagrant up`.
```

You can do this by going to the geoserver/ directory and running ```vagrant up```

In [ ]:
!cd ../geoserver && vagrant status

### Display geoserver status

This should ensure the client can successfully connect to your VM,  if you do not see the Geoserver 'Status' page then something is wrong!

In [ ]:
from IPython.core.display import display, HTML
from geonotebook.config import Config
geoserver = Config().vis_server
display(HTML(geoserver.c.get("/about/status").text))

### Get the rgb.tif file from data.kitware.com

In [ ]:
!wget -O /tmp/rgb.tif https://data.kitware.com/api/v1/file/57b471dd8d777f126827e6b7/download

### Add the layer to the map

In [1]:
M.add_layer("/tmp/rgb.tif")

In [2]:
# List Layers
M.layers

# Select Layer
# M.layers['rgb']

# Get data from layer region - not yet implemented!
# M.layers['rgb'].region 

GeonotebookStack([<GeonotebookLayer('osm_base')>, <GeonotebookLayer('rgb')>])

In [4]:
M.layers['rgb'].data_path

'/tmp/rgb.tif'

In [3]:
M.remove_layer('rgb')

In [4]:
M.layers

GeonotebookStack([<GeonotebookLayer('osm_base')>])

In [ ]:
# Clean up,  delete the workspace
c.delete("/workspaces/test")